In [18]:
#Imports
import ee
#ee.Authenticate()
ee.Initialize() #opt_url='https://earthengine-highvolume.googleapis.com'
import geemap
import geemap.foliumap as geemap
import pandas as pd
import numpy as np
import os
import time

# Map = geemap.Map(center=[40, -100], zoom=4, height=600)
# Map

In [19]:
# MASKING, INDEX CALCULATION, L5 & L7 TO L8 & L9 HARMONIZATION

#  Landsat8 SR scaling function. Applies scaling factors.
def applyScaleFactors(image):
    opticalBands = image.select(['blue', 'green', 'red', 'nir', 'swir1', 'swir2']).multiply(0.0000275).add(-0.2).multiply(10000)
    return image.addBands(opticalBands, None, True) \
                .copyProperties(image)


#  Landsat
def fmask89(region):

    def allMasks89(img):
        # Bit 0 - Fill
        # Bit 1 - Dilated Cloud
        # Bit 2 - Unused
        # Bit 3 - Cloud
        # Bit 4 - Cloud Shadow
        # Bit 5 - Snow
        cloudShadowBitMask = 1 << 3
        cloudsBitMask = 1 << 5
        cirrusBitMask = 1 << 2
        #snowBitMask = 1 << 4
        
        
          # Get the pixel QA band.
        qa = img.select('qa')

          # Flags should be set to zero, indicating clear conditions.
        mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)) \
              .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
              .And(qa.bitwiseAnd(cirrusBitMask).eq(0)) #\
              #.And(qa.bitwiseAnd(snowBitMask).eq(0))

        site = region.buffer(500)

        #  mask the mask with the mask...
        maskedMask = mask.updateMask(mask)

        #  count the number of nonMasked pixels
        maskedCount = maskedMask.select(['qa']) \
            .reduceRegion(reducer=ee.Reducer.count(),
                          geometry=site,
                          scale=ee.Number(30),
                          maxPixels=ee.Number(4e10))

        #  count the total number of pixels
        origCount = img.select(['blue']) \
            .reduceRegion(reducer=ee.Reducer.count(),
                          geometry=site,
                          scale=ee.Number(30),
                          maxPixels=ee.Number(4e10))

        #  calculate the percent of masked pixels
        percent = ee.Number(origCount.get('blue')) \
            .subtract(maskedCount.get('qa')) \
            .divide(origCount.get('blue')) \
            .multiply(100) \
            .round()

        #  Return the masked image with new property and time stamp
        return img.updateMask(mask) \
            .set('CloudSnowMaskedPercent', percent) \
            .set('OrigPixelCount', origCount.get('blue'))\
            .set({'MEAN_SOLAR_AZIMUTH_ANGLE': img.get('SUN_AZIMUTH')})\
            .set({'MEAN_SOLAR_ZENITH_ANGLE': img.get('SUN_ELEVATION')})\
            .set({'Mission': img.get('SPACECRAFT_ID')})\
            .set({'date': img.get('DATE_PRODUCT_GENERATED')})\
            .set({'Scene_ID': img.get('LANDSAT_SCENE_ID')})\
            .copyProperties(img) 

    return allMasks89

# Mask cloud, cloud shadow, and snow in Landsat 5, 7 images
def fmask57(region):

    def allMasks57(img):
        # Bit 0 - Fill
        # Bit 1 - Dilated Cloud
        # Bit 2 - Unused
        # Bit 3 - Cloud
        # Bit 4 - Cloud Shadow
        # Bit 5 - Snow
        cloudShadowBitMask = 1 << 3
        cloudsBitMask = 1 << 5
        cirrusBitMask = 1 << 2
        #snowBitMask = 1 << 4
        
          # Get the pixel QA band.
        qa = img.select('qa')

          # Both flags should be set to zero, indicating clear conditions.
        mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
              .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
              .And(qa.bitwiseAnd(cirrusBitMask).eq(0)) #\
              #.And(qa.bitwiseAnd(snowBitMask).eq(0))

        site = region.buffer(500)

        #  mask the mask with the mask...
        maskedMask = mask.updateMask(mask)

        #  count the number of nonMasked pixels
        maskedCount = maskedMask.select(['qa']) \
            .reduceRegion(reducer=ee.Reducer.count(),
                          geometry=site,
                          scale=ee.Number(30),
                          maxPixels=ee.Number(4e10))

        #  count the total number of pixels
        origCount = img.select(['blue']) \
            .reduceRegion(reducer=ee.Reducer.count(),
                          geometry=site,
                          scale=ee.Number(30),
                          maxPixels=ee.Number(4e10))

        #  calculate the percent of masked pixels
        percent = ee.Number(origCount.get('blue')) \
            .subtract(maskedCount.get('qa')) \
            .divide(origCount.get('blue')) \
            .multiply(100) \
            .round()

        #  Return the masked image with new property and time stamp
        return img.updateMask(mask) \
            .set('CloudSnowMaskedPercent', percent) \
            .set('OrigPixelCount', origCount.get('blue'))\
            .set({'MEAN_SOLAR_AZIMUTH_ANGLE': img.get('SUN_AZIMUTH')})\
            .set({'MEAN_SOLAR_ZENITH_ANGLE': img.get('SUN_ELEVATION')})\
            .set({'Mission': img.get('SPACECRAFT_ID')})\
            .set({'date': img.get('DATE_PRODUCT_GENERATED')})\
            .set({'Scene_ID': img.get('LANDSAT_SCENE_ID')})\
            .copyProperties(img) #.updateMask(ee.Image.constant(1).clip(road).mask().Not())\

    return allMasks57

#
#  Sentinel
#

# Combine s2 and s2 cloudless collections
def get_s2_sr_cld_col(aoi, start_date, end_date, CLOUD_FILTER):
    #  Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    #  Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date))

    #  Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

# Add cloud band to combined S2 collection
def add_cloud_bands(img):
    #  Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    #  Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    #  Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

# Add shadow band to combined S2 collection
def add_shadow_bands(img):
    #  Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    #  Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    #  Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    #  Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10) \
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100}) \
        .select('distance') \
        .mask() \
        .rename('cloud_transform'))

    #  Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    #  Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

# Assemble all cloud and cloud shadow components for final mask
def add_cld_shdw_mask(img):
    #  Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    #  Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    #  Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    #  Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    #  20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20) \
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20}) \
        .rename('cloudmask'))

    #  Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

# # Define a funciton to apply the cloud mask to each image in the collection
# function apply_cld_shdw_mask(img) {
#     # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
#     not_cld_shdw = img.select('cloudmask').Not()

#     #  Subset reflectance bands and update their masks, return the result.
#     return img.select(['B.*', 'qa']).updateMask(not_cld_shdw)
# }

#   Function to mask clouds using the Sentinel-2 QA band
#   @param {ee.Image} image Sentinel-2 image
#   @return {ee.Image} cloud masked Sentinel-2 image

def maskS2clouds(region):
    def allMasksS2(img):

        # Buffer site
        site = region.buffer(500)
        road = ee.FeatureCollection("TIGER/2016/Roads").filterBounds(region.buffer(750)).geometry().buffer(30)
        qa = img.select('QA60')

        #  Bits 10 and 11 are clouds and cirrus, respectively.
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11

        #  Both flags should be set to zero, indicating clear conditions.
        mask = (qa.bitwiseAnd(cloudBitMask).eq(0)) \
           .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

        # # Incorporate s2_cloudless functions
        add_cld_shdw_bands = add_cld_shdw_mask(img)

        not_cld_shdw = add_cld_shdw_bands.select('cloudmask').Not()


        #  mask the mask with the mask...
        maskedMask = mask.updateMask(mask).updateMask(not_cld_shdw)

        #  count the number of nonMasked pixels
        maskedCount = maskedMask.select(['QA60']) \
            .reduceRegion(reducer = ee.Reducer.count(),
                        geometry = site,
                        scale = ee.Number(30),
                        maxPixels = ee.Number(4e10))

        #  count the total number of pixels
        origCount = img.select(['B2']) \
            .reduceRegion(reducer = ee.Reducer.count(),
                          geometry = site,
                          scale= ee.Number(30),
                          maxPixels= ee.Number(4e10))

        #  calculate the percent of masked pixels
        percent = ee.Number(origCount.get('B2')) \
            .subtract(maskedCount.get('QA60')) \
            .divide(origCount.get('B2')) \
            .multiply(100) \
            .round()

        return img.updateMask(mask).select(['B.*', 'QA60']).updateMask(not_cld_shdw) \
            .set('CloudSnowMaskedPercent', percent) \
            .set('OrigPixelCount', origCount.get('B2'))\
            .set({'Mission': img.get('SPACECRAFT_NAME')})\
            .set({'date': img.get('GENERATION_TIME')})\
            .set({'Scene_ID': img.get('GRANULE_ID')})\
            .set({'WRS_PATH': img.get('MGRS_TILE')})\
            .set({'WRS_ROW': img.get('MGRS_TILE')})\
            .copyProperties(img) 

    return allMasksS2


In [20]:
# Function combine, and sort fine and coarse images
def getHLS(startDate, endDate, ls789, ls57, ls5,
              s2_sr_col, s2_cloudless_col, CLOUD_FILTER, 
              landsatBands57, landsatBands89, bandNamesLandsat,S2Bands, bandNamesS2, 
              commonBandNames, region, road):

    if startDate >= '2013-03-18' and endDate >= '2013-03-18':
        #Landsat 8 and Landsat 9
        col789 = ee.ImageCollection(ls789)\
                    .map(lambda image: image.clip(region.buffer(1000)))\
                    .map(fmask89(region)) \
                    .filter(ee.Filter.lte('CloudSnowMaskedPercent', 10))\
                    .filter(ee.Filter.gt('OrigPixelCount', 0))\
                    .map(lambda image: image \
                         .setMulti({\
                             'system:time_start':\
                                 ee.Date(image.date().format('y-M-d'))\
                                 .millis(),\
                             'DOY': image.date().format('D')\
                             })) \
                    .select(commonBandNames) \
                    .map(applyScaleFactors)\
                    .sort('system:time_start') 

        #Sentinel-2

        s2_sr_cloudless = get_s2_sr_cld_col(region, startDate, endDate, CLOUD_FILTER)

        s2 = ee.ImageCollection(s2_sr_cloudless)\
            .filterBounds(region)\
            .map(lambda image: image.clip(col789.first().geometry())) \
            .map(maskS2clouds(region)) \
            .select(S2Bands, bandNamesS2) \
            .filter(ee.Filter.lte('CloudSnowMaskedPercent', 10))\
            .filter(ee.Filter.gt('OrigPixelCount', 0))\
            .map(lambda image: image.resample('bicubic') \
                .setMulti({
                     'system:time_start':
                         ee.Date(image.date().format('y-M-d'))\
                         .millis(),
                     'DOY': image.date().format('D')
                     })) \
            .map(lambda image: image.setDefaultProjection(**{'crs': (col789.first().projection()), 
                                                  'scale': ee.Number(30)})) \
            .select(commonBandNames)\
            .sort('system:time_start')
        
        hls = ee.ImageCollection(col789).merge(ee.ImageCollection(s2)).sort('system:time_start', True)

    elif startDate <= '2013-03-18' and endDate <= '2013-03-18':

        col57 = ee.ImageCollection(ls57)\
                    .filterBounds(region)\
                    .map(lambda image: image.clip(region.buffer(1000)))\
                    .map(fmask89(region)) \
                    .filter(ee.Filter.gt('OrigPixelCount', 0))\
                    .filter(ee.Filter.lte('CloudSnowMaskedPercent', 10))\
                    .map(lambda image: image \
                         .setMulti({\
                             'system:time_start':\
                                 ee.Date(image.date().format('y-M-d'))\
                                 .millis(),\
                             'DOY': image.date().format('D')\
                             })) \
                    .select(commonBandNames) \
                    .map(applyScaleFactors)\
                    .sort('system:time_start') 

        hls = (ee.ImageCollection(col57)).sort('system:time_start', True) 
        
    elif startDate <= '2013-03-18' and endDate >= '2013-03-18': 
        #Landsat 8 and Landsat 9
        col789 = ee.ImageCollection(ls789)\
                    .filterBounds(region)\
                    .map(lambda image: image.clip(region.buffer(1000)))\
                    .map(fmask89(region)) \
                    .filter(ee.Filter.lte('CloudSnowMaskedPercent', 10))\
                    .filter(ee.Filter.gt('OrigPixelCount', 0))\
                    .map(lambda image: image \
                         .setMulti({\
                             'system:time_start':\
                                 ee.Date(image.date().format('y-M-d'))\
                                 .millis(),\
                             'DOY': image.date().format('D')\
                             })) \
                    .select(commonBandNames) \
                    .map(applyScaleFactors)\
                    .sort('system:time_start') 

        #Sentinel-2

        s2_sr_cloudless = get_s2_sr_cld_col(region, startDate, endDate, CLOUD_FILTER)

        s2 = ee.ImageCollection(s2_sr_cloudless)\
            .filterBounds(region)\
            .map(lambda image: image.clip(col789.first().geometry())) \
            .map(maskS2clouds(region)) \
            .select(S2Bands, bandNamesS2) \
            .filter(ee.Filter.lte('CloudSnowMaskedPercent', 10))\
            .filter(ee.Filter.gt('OrigPixelCount', 0))\
            .map(lambda image: image.resample('bicubic') \
                .setMulti({
                     'system:time_start':
                         ee.Date(image.date().format('y-M-d'))\
                         .millis(),
                     'DOY': image.date().format('D')
                     })) \
            .map(lambda image: image.reproject(**{'crs': (col789.first().projection()), 'scale': ee.Number(30)})) \
            .select(commonBandNames)\
            .sort('system:time_start')

        col5 = ee.ImageCollection(ls5)\
                    .filterBounds(region)\
                    .map(lambda image: image.clip(region.buffer(1000)))\
                    .map(fmask89(region)) \
                    .filter(ee.Filter.lte('CloudSnowMaskedPercent', 10))\
                    .filter(ee.Filter.gt('OrigPixelCount', 0))\
                    .map(lambda image: image \
                         .setMulti({\
                             'system:time_start':\
                                 ee.Date(image.date().format('y-M-d'))\
                                 .millis(),\
                             'DOY': image.date().format('D')\
                             })) \
                    .select(commonBandNames) \
                    .map(applyScaleFactors)\
                    .sort('system:time_start') 

        hls = ee.ImageCollection(col789).merge(ee.ImageCollection(s2))\
            .merge(ee.ImageCollection(col5)).sort('system:time_start', True) 
     
    return ee.ImageCollection(hls)
    
  

In [21]:
#Water and PostProc Fxns functions
#DSWE Post Processing
### These functions all go into calculating the USGS Dynamic water m

def AddFmask(image):
    ndvi = ee.Image(image).normalizedDifference(['nir', 'red'])
    nir = ee.Image(image).select(['nir']).multiply(0.0001);  # change DF
    fwater = ndvi.lt(0.01).And(nir.lt(0.11)).Or(ndvi.lt(0.1).And(nir.lt(0.05)))
    fmask = fwater.rename(['fmask'])
    ##mask the fmask so that it has the same footprint as the quality (BQA) band

    return(ee.Image(image).addBands(fmask))

def Mndwi(image):
    return image.normalizedDifference(['green', 'swir1']).rename('mndwi')

def Mbsrv(image):
    return image.select(['green']).add(image.select(['red'])).rename('mbsrv')

def Mbsrn(image):
    return image.select(['nir']).add(image.select(['swir1'])).rename('mbsrn')

def Ndvi(image):
    return image.normalizedDifference(['nir', 'red']).rename('ndvi')

def Awesh(image):
    return (image.addBands(Mbsrn(image)) \
    .expression('blue + 2.5 * green + (-1.5) * mbsrn + (-0.25) * swir2', {
      'blue': image.select(['blue']),
      'green': image.select(['green']),
      'mbsrn': Mbsrn(image).select(['mbsrn']),
      'swir2': image.select(['swir2'])
      }))

def Dswe(i):
    mndwi = Mndwi(i)
    mbsrv = Mbsrv(i)
    mbsrn = Mbsrn(i)
    awesh = Awesh(i)
    swir1 = i.select(['swir1'])
    nir = i.select(['nir'])
    ndvi = Ndvi(i)
    blue = i.select(['blue'])
    swir2 = i.select(['swir2'])

    t1 = mndwi.gt(0.124)
    t2 = mbsrv.gt(mbsrn)
    t3 = awesh.gt(0)
    t4 = mndwi.gt(-0.44) \
    .And(swir1.lt(900)) \
    .And(nir.lt(1500)) \
    .And(ndvi.lt(0.7))
    t5 = mndwi.gt(-0.5) \
    .And(blue.lt(1000)) \
    .And(swir1.lt(3000)) \
    .And(swir2.lt(1000)) \
    .And(nir.lt(2500))

    t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000))

    noWater = t.eq(0) \
    .Or(t.eq(1)) \
    .Or(t.eq(10)) \
    .Or(t.eq(100)) \
    .Or(t.eq(1000))
    hWater = t.eq(1111) \
    .Or(t.eq(10111)) \
    .Or(t.eq(11011)) \
    .Or(t.eq(11101)) \
    .Or(t.eq(11110)) \
    .Or(t.eq(11111))
    mWater = t.eq(111) \
    .Or(t.eq(1011)) \
    .Or(t.eq(1101)) \
    .Or(t.eq(1110)) \
    .Or(t.eq(10011)) \
    .Or(t.eq(10101)) \
    .Or(t.eq(10110)) \
    .Or(t.eq(11001)) \
    .Or(t.eq(11010)) \
    .Or(t.eq(11100))
    pWetland = t.eq(11000)
    lWater = t.eq(11) \
    .Or(t.eq(101)) \
    .Or(t.eq(110)) \
    .Or(t.eq(1001)) \
    .Or(t.eq(1010)) \
    .Or(t.eq(1100)) \
    .Or(t.eq(10000)) \
    .Or(t.eq(10001)) \
    .Or(t.eq(10010)) \
    .Or(t.eq(10100))

    iDswe = noWater.multiply(0) \
    .add(hWater.multiply(1)) \
    .add(mWater.multiply(2)) \
    .add(pWetland.multiply(3)) \
    .add(lWater.multiply(4))

    return(iDswe.rename('dswe'))



##Function for limiting the max number of tasks sent to
#earth engine at one time to avoid time out errors
import time
def maximum_no_of_tasks(MaxNActive, waitingPeriod):
    ##maintain a maximum number of active tasks
    time.sleep(10)
    ## initialize submitting jobs
    ts = list(ee.batch.Task.list())

    NActive = 0
    for task in ts:
        if ('RUNNING' in str(task) or 'READY' in str(task)):
            NActive += 1
    ## wait if the number of current active tasks reach the maximum number
    ## defined in MaxNActive
    while (NActive >= MaxNActive):
        time.sleep(waitingPeriod) # if reach or over maximum no. of active tasks, wait for 2min and check again
        ts = list(ee.batch.Task.list())
        NActive = 0
        for task in ts:
            if ('RUNNING' in str(task) or 'READY' in str(task)):
                NActive += 1
    return()


### Calculuate hillshades to correct DWSE
def CalcHillShades(image, geo):
    MergedDEM = ee.Image("users/eeProject/MERIT").clip(geo.buffer(300))

    hillShade = (ee.Terrain.hillshade(MergedDEM, ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE')),
    image.get('MEAN_SOLAR_ZENITH_ANGLE')).rename(['hillShade']))
    return hillShade

#  ## Calculuate hillshadow to correct DWSE
def CalcHillShadows(image, geo):
    MergedDEM = ee.Image("users/eeProject/MERIT").clip(geo.buffer(3000))
    hillShadow = (ee.Terrain.hillShadow(MergedDEM, ee.Number(image.get('MEAN_SOLAR_AZIMUTH_ANGLE')),
    ee.Number(90).subtract(image.get('MEAN_SOLAR_ZENITH_ANGLE')), 30).rename(['hillShadow']))
    return hillShadow

def removeGeometry(feat):
    #remove geometry of a feature
    
    return ee.Feature(feat).setGeometry(None)

In [22]:
#Master HLS Function
def matchPerSite (site):
    region = ee.Feature(site).geometry()
    
    img_filtered = getHLS(startDate, endDate, ls789, ls57, ls5,
              s2_sr_col, s2_cloudless_col, CLOUD_FILTER, 
              landsatBands57, landsatBands89, bandNamesLandsat,S2Bands, bandNamesS2, 
              commonBandNames, region, road)
    
    return (image_pull(img_filtered, site))

def image_pull(img_filtered, site):
    def pullimg(image):
        sdist = site.geometry().buffer(dist)
  
        lsSample = image

        # mission = ee.String(lsSample.get('SATELLITE')).split('_').get(1)
        imagedate= ee.Date(lsSample.get('system:time_start')) 
        sceneID= lsSample.get('Scene_ID')


        qa = lsSample.select('qa')


        #Create water only mask
        wateronly = water.clip(sdist)
        
        #Update mask on imagery and add Pekel occurrence band for data export.
        waterOut = lsSample.addBands(pekel.select('occurrence'))\
            .updateMask(wateronly)

        def waterOnly(img):
            
            #Band quality thresholds
            white_pixels = 2500 #white pixels
                                 
            #create band theshold masks
            bright_pixels = img.select('red').lt(white_pixels).And(img.select('blue').lt(white_pixels)).And(img.select('green').lt(white_pixels))

            #Mask roads with a 1 pixel buffer of 30m
            road = ee.FeatureCollection('TIGER/2016/Roads').filterBounds(region.buffer(250)).geometry().buffer(30)
            roadMask = ee.Image(0).paint(road,1).Not()
            
            #Collect median elevation for the water pixels where reflectance is extracted
            DEMout = ee.Image("users/eeProject/MERIT").reduceRegion(ee.Reducer.min(), 
                                                                    ee.Feature(site).geometry(), 30);

            f = AddFmask(img).select('fmask')
            d = Dswe(img).select('dswe')
            h = CalcHillShades(img, sdist).select('hillShade')
            hs = CalcHillShadows(img, sdist).select('hillShadow')

            return img.addBands(f).addBands(d).addBands(h).addBands(hs).updateMask(f.lte(2))\
                        .updateMask(d.eq(1).Or(d.eq(2)))\
                        .updateMask(roadMask).updateMask(bright_pixels).toInt16()
           

        waterOut = waterOnly(lsSample)
        
        #reduce images to median and st. dev reflectance
        reducers = ee.Reducer.median().combine(ee.Reducer.stdDev(), "", True)
        
        lsout = waterOut.reduceRegion(reducers, sdist, 30)

        #Collect median elevation for the water pixels where reflectance is extracted
        DEMout = ee.Image("users/eeProject/MERIT").reduceRegion(ee.Reducer.min(), sdist, 30)

        #Create dictionaries of median values and attach them to original site feature.

        output = site.set({
            'date': imagedate,
            'sceneID': sceneID})\
                .set({"blue": lsout.get('blue_median')})\
                .set({"green": lsout.get('green_median')})\
                .set({"red": lsout.get('red_median')})\
                .set({"nir": lsout.get('nir_median')})\
                .set({"swir1": lsout.get('swir1_median')})\
                .set({"swir2": lsout.get('swir2_median')})\
                .set({"qa": lsout.get('qa_median')})\
                .set({"swir1_sd": lsout.get('swir1_stdDev')})\
                .set({"qa_sd": lsout.get('qa_stdDev')})\
                .set({'dswe': lsout.get('dswe_median')})\
                .set({'dswe_sd':lsout.get('dswe_stdDev')})\
                .set({'hillshade':lsout.get('hillShade_median')})\
                .set({'hillshadow':lsout.get('hillShadow_median')})\
                .set({'hillshadow_sd':lsout.get('hillShadow_stdDev')})\
                .set({'azimuth': waterOut.get('MEAN_SOLAR_AZIMUTH_ANGLE')})\
                .set({'zenith': waterOut.get('MEAN_SOLAR_ZENITH_ANGLE')})\
                .set({"pixelCount": waterOut.reduceRegion(ee.Reducer.count(), sdist, 30).get('blue')})\
                .set({'elevation':DEMout.get('elevation')})\
                .set({"blue_sd": lsout.get('blue_stdDev')})\
                .set({"green_sd": lsout.get('green_stdDev')})\
                .set({"red_sd": lsout.get('red_stdDev')})\
                .set({"nir_sd": lsout.get('nir_stdDev')})\
                .set({"swir2_sd": lsout.get('swir2_stdDev')})

        return output

  
    return img_filtered.map(pullimg).filter(ee.Filter.gt('pixelCount', 0))


In [24]:
#Define variables and Collections
#################################
huc1Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc01_5sitelatlong_2')
huc2Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc02_5sitelatlong_2')
huc3Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
huc4Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
huc5Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc05_5sitelatlong_2')
huc6Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc06_5sitelatlong_2')
huc7Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc07_5sitelatlong_2')
huc8Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc08_5sitelatlong_2')
huc9Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc09_5sitelatlong_2')
huc10Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc10_5sitelatlong_2')
huc11Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc11_5sitelatlong_2')
huc12Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc12_5sitelatlong_2')
huc13Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc13_5sitelatlong_2')
huc14Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc14_5sitelatlong_2')
huc15Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc15_5sitelatlong_2')
huc16Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc16_5sitelatlong_2')
huc17Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
huc18Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc18_5sitelatlong_2')
huc19Sites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc19_5sitelatlong_2')

#define sites of interst
#site = \
#    .map(lambda feat: feat.set({'SiteID': ee.Feature(feat).get('\ufeffSiteID')}))
#feature = ee.List([ee.Feature(ee.Geometry.Point(-148.8177778, 69.0158333), {'SiteID': 'Sag_Gauge'})])
site = ee.FeatureCollection(huc10Sites)
siteString = ee.String('huc10Sites')
print(siteString.getInfo())
print(site.size().getInfo())
#print(site.getInfo())


region = site.first().geometry().centroid()

dist = 500
threshold = 80

#define data temporal bounds
startDate = '2013-03-18'
endDate = '2014-04-23'

# startDate = '2013-03-18'
# endDate = '2023-03-04'

# startDate = '2000-01-01'
# endDate = '2023-03-04'

startDate_ls7 = '2000-01-01'
endDate_ls7 = '2017-03-18'#'2017-03-18'

print(startDate, endDate)

#Define collections/  iables needed
commonBandNames = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'qa'])

# landsat band names including qa band for masking
bandNamesLandsat = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'qa'])
landsatBands57 = ee.List(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'])
landsatBands89 = ee.List(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'])

#Sentinel Band Names including qa band for masking
S2Bands = ['B2','B3','B4','B5','B6','B7', 'QA60']
bandNamesS2 = ['blue','green','red','nir','swir1','swir2', 'qa']

ls9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")\
        .filterDate(startDate, endDate) \
        .filterBounds(site.geometry())\
        .select(landsatBands89, bandNamesLandsat)

ls8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        .filterDate(startDate, endDate) \
        .filterBounds(site.geometry())\
        .select(landsatBands89, bandNamesLandsat)

ls7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')\
                    .filterBounds(site.geometry())\
                    .filterDate(startDate_ls7, endDate_ls7)\
                    .select(landsatBands57, bandNamesLandsat)

ls789 = ee.ImageCollection(ls9.merge(ls8).merge(ls7))                  

ls5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')\
        .filterDate(startDate, endDate) \
        .filterBounds(site.geometry())\
        .select(landsatBands57, bandNamesLandsat)


ls57 = ee.ImageCollection(ls7.merge(ls5))

s2_sr_col = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
        .filterDate(startDate, endDate) \
        .filterBounds(site.geometry())

s2_cloudless_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

#S2 cloudless parameters
CLOUD_FILTER = 80
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 75


#Other useful datasets (add pekel here etc)
road = ee.FeatureCollection("TIGER/2016/Roads")
pekel = ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
water = pekel.select('occurrence').gt(threshold)
water = water.updateMask(water)




# #The harmonization function to get raw images and test on a single point if needed
# hls = getHLS(startDate, endDate, ls789, ls57, ls5,
#               s2_sr_col, s2_cloudless_col, CLOUD_FILTER, 
#               landsatBands57, landsatBands89, bandNamesLandsat,S2Bands, bandNamesS2, 
#               commonBandNames, region, road)

# print(hls.limit(2).getInfo())

#Function Call
output = site.map(matchPerSite).flatten().map(removeGeometry)

#print(output.limit(3).getInfo())

#TO EXPORT TABLE
task = ee.batch.Export.table.toDrive(
    collection = ee.FeatureCollection(output),
    description = ee.String(siteString).getInfo() +'_'+ ee.String(startDate).getInfo() + '_' + ee.String(endDate).getInfo(),
    folder = 'HLS_Pull',
    fileFormat = 'csv',
    selectors = ['SiteID','date','sceneID','blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'qa', 'dswe',
   'blue_sd', 'green_sd', 'red_sd', 'nir_sd', 'swir1_sd', 'swir2_sd', 'qa_sd', 'dswe_sd',
   'hillshade','hillshadow','hillshadow_sd','azimuth','zenith','pixelCount','elevation'])

import time
#maximum_no_of_tasks(10, 10)
#task.start()


huc10Sites
124
2013-03-18 2014-04-23


In [93]:
#Function Call
output = site.map(matchPerSite).flatten().map(removeGeometry)

#print(output.limit(3).getInfo())

#TO EXPORT TABLE
task = ee.batch.Export.table.toDrive(
    collection = ee.FeatureCollection(output),
    description = ee.String(siteString).getInfo() +'_'+ ee.String(startDate).getInfo() + '_' + ee.String(endDate).getInfo(),
    folder = 'HLS_Pull',
    fileFormat = 'csv',
    selectors = ['SiteID','date','sceneID','blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'qa', 'dswe',
   'blue_sd', 'green_sd', 'red_sd', 'nir_sd', 'swir1_sd', 'swir2_sd', 'qa_sd', 'dswe_sd',
   'hillshade','hillshadow','hillshadow_sd','azimuth','zenith','pixelCount','elevation'])

import time
maximum_no_of_tasks(10, 10)
task.start()


In [36]:
import geemap.foliumap as geemap
#center=[40.5645116, -79.7725485]
#center=[40.6525658, -80.3703416]
Map = geemap.Map(center=[40.411829338, -87.0362363063589], zoom=10, height=600)
region = hls.first().geometry().centroid()
road = ee.FeatureCollection('TIGER/2016/Roads')

visualization = {
    'min': 0.0,
    'max': 4000,
    'gamma': 1.4,
    'bands': ['blue', 'green', 'red']}
road = ee.FeatureCollection('TIGER/2016/Roads').filterBounds(region.buffer(300)).geometry().buffer(30)
roadMask = ee.Image(0).paint(road,1).Not()


Map.addLayer(hls.first().updateMask(roadMask), visualization, 'hls first')
Map.centerObject(region, zoom = 11)
Map.addLayer(region.buffer(200))
Map

In [ ]:
import ee

# Define the function that loads and filters the image collection
def load_and_filter_image_collection():
    # Initialize the Earth Engine instance
    ee.Initialize()
    
    # Load the Landsat 8 image collection
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
    
    # Filter the collection by date and cloud cover
    filtered_collection = collection.filterDate('2020-01-01', '2020-12-31')\
        .filterBounds(ee.Geometry.Point([-122.262, 37.8719]).buffer(1000))\
        .filterMetadata('CLOUD_COVER', 'less_than', 10)
    
    # Reset the Earth Engine instance
    ee.Reset()
    
    # Return the filtered image collection
    return filtered_collection

# Define the function that runs algorithms on the filtered image collection and exports the results
def run_algorithms_and_export_results(filtered_collection):
    # Initialize the Earth Engine instance
    ee.Initialize()
    
    # Map the NDVI algorithm over the filtered image collection
    ndvi_collection = filtered_collection.map(lambda image: image.normalizedDifference(['B5', 'B4']))
    
    # Export the NDVI collection as a GeoTIFF file to Google Drive
    task = ee.batch.Export.image.toDrive(image=ndvi_collection.toBands(), description='NDVI', folder='earth_engine', scale=30)
    #task.start()
    
    # Reset the Earth Engine instance
    ee.Reset()

# Call the functions in sequence
filtered_collection = load_and_filter_image_collection()
run_algorithms_and_export_results(filtered_collection)